### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from tqdm.notebook import tqdm
from neuropy.core import Epoch
from neuropy.utils.mathutil import min_max_scaler
from neuropy import plotting
from scipy import stats
import pandas as pd
import subjects

### Signficant sequences(wcorr) compare SD vs NSD 

In [ ]:
sessions = subjects.pf_sess()

In [ ]:
replay_df = []
for sub, sess in enumerate(sessions):

    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    zt_starts = np.array([0,2.5,5])
    zt_stops = np.array([2.5,5,7.5])
    epochs = np.vstack((zt_starts,zt_stops)).T* 3600 + post[0]
    epochs = np.insert(epochs,0,maze,axis=0)
    labels = ['MAZE','0-2.5','2.5-5','5-7.5']

    for i, (t1,t2) in enumerate(epochs):
        replay_pbe = sess.replay_pbe.time_slice(t1,t2).to_dataframe()
        for direction in ["up", "down"]:
            perc = replay_pbe[direction + "_percentile_score"].values
            is_replay = (perc <= 5) | (perc >= 95)
            scores = replay_pbe[direction + "_score"].values[is_replay]
            order = replay_pbe[direction + "_replay_order"].values[is_replay]
            order, counts = np.unique(order, return_counts=True)
            df = pd.DataFrame(
                dict(
                    zt=labels[i],
                    replay_order=["r"],
                    prop=counts.sum() / len(replay_pbe),
                    # scores=np.abs(scores),
                    animal=sub,
                    grp=sess.tag,
                )
            )
            replay_df.append(df)

replay_df = pd.concat(replay_df, ignore_index=True)

# subjects.GroupData().save(replay_df,'replay_sig_frames')

In [ ]:

_,ax = plt.subplots() 
sns.barplot(data=replay_df, x="zt", y="prop",hue='grp')


### Distribution of wcorr SD vs NSD

In [ ]:
sessions = subjects.pf_sess()

In [ ]:
print(f"#Sessions={len(sessions)}")
replay_df = []
for sub, sess in enumerate(sessions):

    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    replay_pbe = pd.concat(
        [sess.pre_replay_pbe.to_dataframe(), sess.replay_pbe.to_dataframe()],
        ignore_index=True,
    )

    metadata = [sess.pre_replay_pbe.metadata, sess.replay_pbe.metadata]

    wcorr_up = replay_pbe["up_score"].values
    up_shuffle = np.hstack([_["up_shuffle_score"] for _ in metadata])

    wcorr_down = -replay_pbe["down_score"].values
    down_shuffle = -np.hstack([_["down_shuffle_score"] for _ in metadata])

    best_bool = np.abs(wcorr_up) > np.abs(wcorr_down)

    best_wcorr = np.zeros(len(wcorr_up))
    best_wcorr[best_bool] = wcorr_up[best_bool]
    best_wcorr[~best_bool] = wcorr_down[~best_bool]
    best_shuffle = np.zeros_like(up_shuffle)
    best_shuffle[:, best_bool] = up_shuffle[:, best_bool]
    best_shuffle[:, ~best_bool] = down_shuffle[:, ~best_bool]

    percentile = np.array(
        [
            stats.percentileofscore(best_shuffle[:, i], best_wcorr[i], kind="strict")
            for i in range(len(best_wcorr))
        ]
    )

    abs_score = np.abs(best_wcorr)
    abs_shuffle_score = np.abs(best_shuffle)
    mean_shuffle_score = abs_shuffle_score.mean(axis=0)
    std_shuffle_score = abs_shuffle_score.std(axis=0)
    seq_score = (abs_score - mean_shuffle_score) / std_shuffle_score

    zt_starts = np.array([0, 2.5, 5])
    zt_stops = np.array([2.5, 5, 7.5])
    epochs = np.vstack((zt_starts, zt_stops)).T * 3600 + post[0]
    epochs = np.insert(epochs, 0, maze, axis=0)
    epochs = np.insert(epochs, 0, pre, axis=0)
    labels = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]

    pval = np.zeros(len(best_wcorr))
    positive_bool =best_wcorr <= 0 
    pval[positive_bool] = percentile[positive_bool] / 1000
    pval[~positive_bool] = 1 - percentile[~positive_bool] / 1000

    for i, (t1, t2) in enumerate(epochs):
        indx = (replay_pbe.start > t1) & (replay_pbe.start < t2)
        df = pd.DataFrame(
            dict(
                zt=labels[i],
                perc=percentile[indx],
                seq_score=seq_score[indx],
                pval=pval[indx],
                wcorr=best_wcorr[indx],
                animal=sub,
                grp=sess.tag,
            )
        )
        replay_df.append(df)

replay_df = pd.concat(replay_df, ignore_index=True)

# subjects.GroupData().save(replay_df, "replay_wcorr")


In [ ]:
from statannotations.Annotator import Annotator
from subjects import stat_kw

_, axs = plt.subplots(2, 3)
axs = axs.reshape(-1)
colors = subjects.colors_sd(1)
df = replay_df[replay_df.pval < 0.05]

for i, t in enumerate(labels):
    sns.histplot(
        data=df[df.zt == t],
        x="pval",
        hue="grp",
        common_bins=True,
        common_norm=False,
        stat="probability",
        ax=axs[i]
    )
# for g,grp in enumerate(['NSD','SD']):
# df_new =df[df['grp']==grp]
# plot_kw =dict(data=df,x='zt',y='pval',hue='grp',ax=ax,palette=subjects.colors_sd(1))
# sns.violinplot(**plot_kw,showfliers=False,split=True,cut=True,scale='width')
# ax.grid(axis='y',zorder=-1)

# orders = replay_df.zt.unique()
# pairs = [('PRE', _) for _ in orders[1:]]
# annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
# annotator.configure(test="t-test_welch", **stat_kw)
# annotator.apply_and_annotate()
# annotator.reset_configuration()


# _,axs = plt.subplots(1,5)
# for i,t in enumerate(labels):

#     df =replay_df[replay_df.zt==t]
#     d1 = df[df.grp=='NSD'].seq_score.values
#     d2 = df[df.grp=='SD'].seq_score.values
#     sns.ecdfplot(data=df, x="perc",hue='grp',ax=axs[i],lw=1)
#     print(stats.kstest(d1,d2))


### wcorr vs jump distance

In [ ]:
sessions = subjects.pf_sess()

In [ ]:
get_jump_dist = lambda arr: np.diff(np.argmax(arr, axis=0)).mean()
wcorr_jump_df = []
for sub, sess in enumerate(sessions):

    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    zt_starts = np.array([0, 2.5, 5])
    zt_stops = np.array([2.5, 5, 7.5])
    epochs = np.vstack((zt_starts, zt_stops)).T * 3600 + post[0]
    epochs = np.insert(epochs, 0, maze, axis=0)
    labels = ["MAZE", "0-2.5", "2.5-5", "5-7.5"]

    replay_pbe = sess.replay_pbe.to_dataframe()

    for direction in ["up", "down"]:
        wcorr = np.abs(replay_pbe[direction + "_score"].values)
        posteriors = sess.replay_pbe.metadata[direction + "_posterior"]
        dx = 1 / posteriors[0].shape[0]
        jump_dist = np.array([get_jump_dist(_) for _ in posteriors]) * dx
        for i, (t1, t2) in enumerate(epochs):
            indx = (replay_pbe.start > t1) & (replay_pbe.start < t2)

            df = pd.DataFrame(
                dict(
                    zt=labels[i],
                    wcorr=wcorr[indx],
                    jump=np.abs(jump_dist[indx]),
                    animal=sub,
                    grp=sess.tag,
                )
            )
            wcorr_jump_df.append(df)

wcorr_jump_df = pd.concat(wcorr_jump_df, ignore_index=True)

# subjects.GroupData().save(replay_df,'replay_wcorr')


In [ ]:
_,ax = plt.subplots(1,1)
sns.violinplot(data=wcorr_jump_df,x='zt',y='jump',hue='grp',ax=ax)
# sns.displot(data=wcorr_jump_df,x='jump',y='wcorr',binwidth=(0.05,0.05),col='grp')

# _, axs = plt.subplots(1, 4)
# for i, t in enumerate(labels):

#     df = wcorr_jump_df[wcorr_jump_df.zt == t]
#     d1 = df[df.grp=='NSD'].jump.values
#     d2 = df[df.grp=='SD'].jump.values
#     sns.barplot(
#         data=df,
#         x='zt',
#         y="jump",
#         hue="grp",
#         ax=axs[i],
#         # lw=1,
#         # common_norm=False,
#         # common_bins=True,
#         # stat="probability",
#         # fill=False,
#         # element='poly',
#         # cumulative=True,
#     )
#     print(stats.kstest(d1,d2))


### Signficant sequences histogram radon transform

In [ ]:
sessions = subjects.nsd.pf_sess

In [ ]:
replay_df = []
for sub, sess in enumerate(sessions):

    post = sess.paradigm["post"].flatten()
    replay_pbe = sess.replay_pbe.to_dataframe()
    is_replay = replay_pbe.is_replay
    velocity_bool = np.abs(replay_pbe.velocity.values) > 100

    sig_events = np.logical_and(is_replay, velocity_bool)
    starts = replay_pbe.start.values[sig_events]
    bins = np.arange(post[0], post[0] + 2 * 3600, 600)
    hist_replay = np.histogram(starts, bins)[0]
    replay_df.append(
        pd.DataFrame(
            {
                "bins": ((bins[:-1] - post[0]) / 3600).round(2),
                "hist": hist_replay,
                "grp": sess.tag,
            }
        )
    )

replay_df = pd.concat(replay_df, ignore_index=True)


In [ ]:

fig = plotting.Fig(grid=(2, 2))
ax = fig.subplot(fig.gs[0])
sns.barplot(data=replay_df, x="bins", y="hist", hue="grp")


### Decoding visualization

In [ ]:
from hfuncs import plot_in_bokeh
import bokeh.plotting as bplot

sess = subjects.nsd.ratSday2[0]
post = sess.paradigm["post"].flatten()
replay_pbe = sess.replay_pbe.to_dataframe()
starts = sess.replay_pbe.starts
wcorr, scores, posteriors = [], [], []
for i, d in enumerate(["up", "down"]):
    scores.append(replay_pbe[d + "_percentile_score"].values)
    posteriors.extend(sess.replay_pbe.metadata[d + "_posterior"])
    wcorr.append(replay_pbe[d + "_score"].values)

wcorr = np.concatenate(wcorr)
scores = np.concatenate(scores)
starts = np.tile(starts, (2,)) - post[0]
scores_color = np.where((scores >= 95) | (scores <= 5), scores, 50)

n_epochs = 7000
annotate_dict = dict(evt=np.arange(n_epochs), wcorr=wcorr[:n_epochs])
bplot.output_notebook()
# bplot.output_file(subjects.figpath_sd/'test_wake_decoding.html')
p = plot_in_bokeh(
    starts[:n_epochs]/3600,
    scores[:n_epochs],
    [posteriors[_] for _ in range(n_epochs)],
    annotate=annotate_dict,
    color_by=scores_color[:n_epochs],
    palette="jet",
    size=8,
)
# # p.line(pos.time,pos.x/350 + 0.2,line_width=2,color='black')
bplot.show(p)


### Examples for figure4 sd paper

In [ ]:
sessions = subjects.nsd.ratSday2 + subjects.sd.ratSday3

# events= [[1221,2836,5072,2381,5179],[1326,1372,1824,1292,1516]]
events= [[316,164,1416,3310,4201,3642,6728,6868],[263,148,2078,1900,3859,3468,6861,5978]]


In [ ]:
replay_example_dict = {}
for sub, sess in enumerate(sessions):
    replay = sess.replay_pbe
    replay_df = sess.replay_pbe.to_dataframe()
    starts = sess.replay_pbe.starts
    wcorr, scores, posteriors = [], [], []
    for i, d in enumerate(["up", "down"]):
        scores.append(replay_df[d + "_percentile_score"].values)
        posteriors.extend(replay.metadata[d + "_posterior"])
        wcorr.append(replay_df[d + "_score"].values)
    wcorr = np.concatenate(wcorr)
    replay_example_dict[sess.tag] = dict(
        posteriors=[posteriors[p] for p in events[sub]], wcorr=wcorr[events[sub]]
    )

subjects.GroupData().save(replay_example_dict,'replay_examples')

### Mean replay score across all events as a function of time 

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d

sessions = subjects.nsd.ratUday2 + subjects.sd.ratUday4

In [ ]:
from scipy import stats

replay_df = []
for sub, sess in enumerate(sessions):

    post = sess.paradigm["post"].flatten()
    replay_pbe = sess.replay_pbe.to_dataframe()
    starts = sess.replay_pbe.starts
    replay_score = replay_pbe.score
    bins = np.arange(post[0], post[0] + 2 * 3600, 600)

    mean_score = stats.binned_statistic(starts, replay_score, bins=bins)[0]
    replay_df.append(
        pd.DataFrame(
            {
                "bins": ((bins[:-1] - post[0]) / 3600).round(2),
                "hist": mean_score,
                "grp": sess.tag,
            }
        )
    )

replay_df = pd.concat(replay_df, ignore_index=True)


In [ ]:
from neuropy import plotting
import seaborn as sns

fig = plotting.Fig(grid=(2, 2))
ax = fig.subplot(fig.gs[0])
sns.lineplot(data=replay_df, x="bins", y="hist", hue="grp")


### Distribution of replay scores and comparison with shuffled scores

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d

sessions = subjects.nsd.ratUday2 + subjects.sd.ratUday4

In [ ]:
from scipy import stats

score_df = []
for sub, sess in enumerate(sessions):

    post = sess.paradigm["post"].flatten()
    replay_pbe = sess.replay_pbe.to_dataframe()

    starts = sess.replay_pbe.starts
    replay_score = replay_pbe.score

    shuffle_scores = sess.replay_pbe.metadata["shuffle_score"].flatten()
    shuffle_starts = np.repeat(starts, 200)

    bins = np.arange(2) * 3600 + post[0]
    replay_df = pd.DataFrame(
        {
            "Zt": np.digitize(starts, bins),
            "starts": starts,
            "score": replay_score,
            "grp": sess.tag,
        }
    )
    shuffle_df = pd.DataFrame(
        {
            "Zt": np.digitize(shuffle_starts, bins),
            "starts": shuffle_starts,
            "score": shuffle_scores,
            "grp": f"{sess.tag}_shuffle",
        }
    )
    score_df.append(pd.concat([replay_df, shuffle_df]))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
sess.replay_pbe.metadata["shuffle_score"].shape


In [ ]:
from neuropy import plotting
import seaborn as sns

fig = plotting.Fig(grid=(2, 2))
ax = fig.subplot(fig.gs[0])
sns.violinplot(data=score_df, x="Zt", y="score", hue="grp")


### Posterior time lag comparison SD vs NSD

In [ ]:
sessions = subjects.nsd.ratUday2 + subjects.sd.ratUday4

In [ ]:
import scipy.signal as sg
from scipy.ndimage import gaussian_filter1d

lag_time_all = []
for sub,sess in enumerate(sessions):
    post = sess.paradigm['post'].flatten()
    starts = sess.replay_pbe.starts
    # ind = (starts>=post[0]+2.5*3600) &(starts<=post[0]+5*3600)
    ind = starts<post[0]

    posteriors = sess.replay_pbe.metadata['up_posterior']
    posteriors = [posteriors[_] for _ in np.argwhere(ind).squeeze()] 
    posteriors = np.hstack(posteriors)

    npos,nt = posteriors.shape
    rows,cols = np.tril_indices(npos,-1)
    lags = sg.correlation_lags(nt,nt,mode='same')*0.02
    idx = (lags>=-0.5) & (lags<=0.5)
    lags = lags[idx]

    lag_time=np.zeros((npos,npos))
    for p1 in range(npos):
        for p2 in range(npos):
            xcorr = sg.correlate(posteriors[p1],posteriors[p2],mode='same')[idx]
            lag_time[p1,p2] = lags[gaussian_filter1d(xcorr,sigma=0.06/0.02).argmax()]

    lag_time_all.append(lag_time)

In [ ]:
_,axs = plt.subplots(2,2)

for i,lag_mat in enumerate(lag_time_all):

    ax = axs[0,i]
    im = ax.pcolormesh(lag_mat,cmap='bwr',vmin=-0.4,vmax=0.4)
    cb = plt.colorbar(im,ax=ax)

    ax2 = axs[1,i]
    im = ax2.pcolormesh(np.abs(lag_mat),cmap='jet',vmin=0,vmax=0.4)
    cb = plt.colorbar(im,ax=ax2)


### Which positions are being replayed the most in POST

In [ ]:
sessions = subjects.nsd.ratUday2 + subjects.sd.ratUday4

In [ ]:
import scipy.signal as sg
from scipy.ndimage import gaussian_filter1d

mean_posterior = []
for sub,sess in enumerate(sessions):
    post = sess.paradigm['post'].flatten()
    starts = sess.replay_pbe.starts
    ind = (starts>=post[0]+2.5*3600) &(starts<=post[0]+5*3600)

    posteriors = sess.replay_pbe.metadata['up_posterior']
    posteriors = [posteriors[_] for _ in np.argwhere(ind).squeeze()] 
    mean_posterior.append(np.hstack(posteriors).mean(axis=1))



In [ ]:
_,axs = plt.subplots(1,2,sharey=True)

for i,p in enumerate(mean_posterior):
    axs[i].plot(p)


### Forward and reverse replay during POST SD vs NSD 

In [ ]:
sessions = subjects.pf_sess()

In [ ]:
from replay_funcs import get_jump_distance
replay_df = []
jd_df = []
for sub, sess in enumerate(sessions):
    maze = sess.paradigm['maze'].flatten()
    post = sess.paradigm['post'].flatten()
    replay_pbe_df = sess.replay_pbe.to_dataframe()
    metadata = sess.replay_pbe.metadata

    wcorr_up = replay_pbe_df["up_score"].values
    seq_score_up = replay_pbe_df["up_sequence_score"].values
    percentile_up = replay_pbe_df["up_percentile_score"].values
    jd_up = get_jump_distance(metadata['up_posterior'],'max')


    wcorr_down = -replay_pbe_df["down_score"].values
    seq_score_down = replay_pbe_df["down_sequence_score"].values
    percentile_down = 100-replay_pbe_df["down_percentile_score"].values
    jd_down = get_jump_distance(metadata['down_posterior'],'max')


    best_bool = np.abs(wcorr_up) > np.abs(wcorr_down)

    wcorr = np.zeros(len(replay_pbe_df))
    wcorr[best_bool] = wcorr_up[best_bool]
    wcorr[~best_bool] = wcorr_down[~best_bool]

    seq_score = np.zeros(len(replay_pbe_df))
    seq_score[best_bool] = seq_score_up[best_bool]
    seq_score[~best_bool] = seq_score_down[~best_bool]

    percentile = np.zeros(len(replay_pbe_df))
    percentile[best_bool] = percentile_up[best_bool]
    percentile[~best_bool] = percentile_down[~best_bool]

    jd= np.zeros(len(replay_pbe_df))
    jd[best_bool] = jd_up[best_bool]
    jd[~best_bool] = jd_down[~best_bool]



    is_forward = (wcorr > 0) & ((percentile < 2.5) | (percentile > 97.5))
    is_reverse = (wcorr < 0) & ((percentile < 2.5) | (percentile > 97.5))

    zt_starts = np.array([0, 2.5, 5])
    zt_stops = np.array([2.5, 5, 7.5])
    epochs = np.vstack((zt_starts, zt_stops)).T * 3600 + post[0]
    epochs = np.insert(epochs, 0, maze, axis=0)
    labels = ["MAZE", "0-2.5", "2.5-5", "5-7.5"]

    for i, (t1, t2) in enumerate(epochs):
        indx = (replay_pbe_df.start > t1) & (replay_pbe_df.start < t2)
        n_pbes = np.count_nonzero(indx)
        df = pd.DataFrame(
            dict(
                zt=labels[i],
                replay_order=["forward", "reverse"],
                prop=[
                    np.count_nonzero(is_forward[indx]) / n_pbes,
                    np.count_nonzero(is_reverse[indx]) / n_pbes,
                ],
                jd=jd[indx].mean(),
                animal=sub,
                grp=sess.tag,
            )
        )
        replay_df.append(df)

replay_df = pd.concat(replay_df, ignore_index=True)

# subjects.GroupData().save(replay_df, "replay_order")


In [ ]:
# _,ax = plt.subplots(1,1)
# sns.barplot(data=replay_df,x='zt',y='perc',hue='grp',ax=ax, ci =99)

_, axs = plt.subplots(1, 2,sharey=True)

for i, odr in enumerate(["forward", "reverse"]):
    ax = axs[i]
    df = replay_df[replay_df["replay_order"] == odr]
    sns.boxplot(
        data=df,
        x="zt",
        y="prop",
        hue="grp",
        ax=ax,
        # split=True,
        showfliers=False,
        # estimator=np.median,
        palette=subjects.colors_sd(1.4),
        dodge=True,
        # join=False,
        # width='area',
        # stat="probability",
        # common_norm=False,
        # cumulative=True,
        # showfliers=False,
    )
    # print(stats.ttest_ind(d1, d2,alternative='greater'))
    ax.set_title(odr)
    ax.set_ylim([0,0.3])


# ax = axs[2]
# sns.boxplot(
#     data=df,
#     x="zt",
#     y="jd",
#     hue="grp",
#     ax=ax,
#     # split=True,
#     showfliers=False,
#     # estimator=np.median,
#     palette=subjects.colors_sd(1.4)
#     # dodge=True,
#     # join=False,
#     # width='area',
#     # stat="probability",
#     # common_norm=False,
#     # cumulative=True,
#     # showfliers=False,
# )
# # print(stats.ttest_ind(d1, d2,alternative='greater'))
# ax.set_title(odr)
# # ax.set_ylim([0,0.3])
